In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#input_file = "output_20200219_2125PM.csv" #complete
#input_file = "output_20191116_1256PM.csv" #partial
input_file = "output_20200219_2125PM.csv" #double
df = pd.read_csv(input_file, header = 0)
df['FLOW'] = df['SRCNODE']+'-'+df['DESTNODE']
df['FM']=df['FAILURE']+df['MISSING']
df_failure = df[ (df.FM==1)]

In [3]:
df_dummy = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW','FM','RETRIES','FILESIZE', 'THROUGHPUT','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S','FLOW'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW'])
df_dummy['THROUGHPUT']=df_dummy['THROUGHPUT'].str[0:-4]
df_dummy['THROUGHPUT']=pd.to_numeric(df_dummy['THROUGHPUT'], errors='coerce')
df_dummy = df_dummy.replace(np.nan, 0, regex=True)
y_complete=df_dummy['LABEL']
X_complete=df_dummy.drop(['LABEL'],axis=1)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [5]:
df_dummy_cate = pd.get_dummies(df[['FLOW','FM','LABEL']], prefix=['FLOW'], columns=['FLOW'])
#df_dummy_cate = pd.get_dummies(df_failure[['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW','RETRIES','LABEL']], prefix=['SRC', 'DEST','SRC_R','DEST_S','FLOW'], columns=['SRCNODE', 'DESTNODE','SRC_R','DEST_S','FLOW'])
y_cate=df_dummy_cate['LABEL']
X_cate=df_dummy_cate.drop(['LABEL'],axis=1)
X_cate[0:1]

,FM,FLOW_CacheCal-CacheFNAL,FLOW_CacheCal-CacheNY,FLOW_CacheCal-CacheSDSC,FLOW_CacheCal-CacheUC,FLOW_CacheCal-CacheUNL,FLOW_CacheCal-OriginCal,FLOW_CacheCal-OriginFNAL,FLOW_CacheCal-OriginNCSA,FLOW_CacheCal-OriginSDSC,...,FLOW_OriginUNL-CacheFNAL,FLOW_OriginUNL-CacheNY,FLOW_OriginUNL-CacheSDSC,FLOW_OriginUNL-CacheUC,FLOW_OriginUNL-CacheUNL,FLOW_OriginUNL-OriginCal,FLOW_OriginUNL-OriginFNAL,FLOW_OriginUNL-OriginNCSA,FLOW_OriginUNL-OriginSDSC,FLOW_OriginUNL-OriginUC
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

In [7]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct_class={}
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.LABEL==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['LABEL'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
            correct_class[label]=1
        else:
            correct_class[label]=0
    return correct_class, correct, correct/num 

In [8]:
def print_s_c(c):
    s_c=sorted(c.items(), key=lambda x: x[1], reverse=True)
    num_link=0;num_host=0;link_pos=0;host_pos=0
    for i in s_c:
        name=i[0]
        flag=i[1]
        if 'Link' in name:
            num_link+=1
            if flag==1:
                link_pos+=1
        else:
            num_host+=1
            if flag==1:
                host_pos+=1
    return link_pos*1.0/num_link,host_pos*1.0/num_host

In [9]:
training_dataset={"complete":{"X":X_complete,
                 "y":y_complete},
                 "cate":{"X":X_cate,
                 "y":y_cate}}

In [10]:
def train_dt(classifier):
    df=[df_dummy,df_dummy_cate]
    i=0
    for k, d in training_dataset.items():
        X=d["X"]
        y=d['y']
        clf = classifier
        clf_model=clf.fit(X,y)
        pred=clf.predict(X)
        balanced_accu=balanced_accuracy_score(y, pred)
        f1=f1_score(y, pred,average='weighted')
        print(str(k)+":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df[i], clf.classes_,j)
            print("Top-"+str(j)+" Accu="+str(accu)) 
        i=i+1

In [11]:
clf_mn = MultinomialNB()
clf_gs = GaussianNB()
clf_bn=BernoulliNB()
train_dt(clf_bn)

complete:balanced_accu=0.3020164177008118:F1-Score=0.34701398337066414
Top-1 Accu=0.6515151515151515
Top-2 Accu=0.7121212121212122
Top-3 Accu=0.7575757575757576
cate:balanced_accu=0.014670368138393093:F1-Score=0.05329688590665327
Top-1 Accu=0.014492753623188406
Top-2 Accu=0.043478260869565216
Top-3 Accu=0.07246376811594203


In [84]:
%%time
clf = MultinomialNB()
clf_model=clf.fit(X_complete,y_complete)

CPU times: user 788 ms, sys: 35.2 ms, total: 823 ms
Wall time: 220 ms


In [12]:
kernel = 1.0 * RBF([1.0])
gpc_rbf_isotropic = GaussianProcessClassifier(kernel=kernel).fit(X_complete, y_complete)

KeyboardInterrupt: 

In [ ]:
kernel = 1.0 * RBF([1.0, 1.0])
gpc_rbf_anisotropic = GaussianProcessClassifier(kernel=kernel).fit(X_complete, y_complete)

In [ ]:
train_dt(gpc_rbf_isotropic)